In [37]:
import pandas as pd

In [38]:
df=pd.read_csv('train/train.csv')



In [39]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [40]:
df=df.dropna()


In [41]:
X = df.drop('label',axis=1)

In [42]:
y = df['label']

In [43]:
X.shape

(18285, 4)

In [44]:
y.shape

(18285,)

In [45]:
import tensorflow as tf

In [46]:
tf.__version__

'2.1.0'

In [47]:
from tensorflow import keras
from keras.layers import Embedding, Dense, LSTM, Dropout
from keras.models import Sequential
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences

## Text Preprocessing

In [48]:
import nltk
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()


## For learns => learn type

In [49]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\yyash\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [50]:
messages=X.copy()
messages.reset_index(inplace=True)


In [51]:
corpus = []
for i in range(len(messages)):
    review = re.sub('[^a-zA-Z]'," ",messages['title'][i])
    review = review.lower()
    review = review.split()
    review = [ps.stem(word) for word in review if not word in stopwords.words('english')]
    review = " ".join(review)
    corpus.append(review)

#### One hot and Pad sequence

In [52]:
## Vocab
vocab = 5000

In [53]:
one_hot_rep = [one_hot(words,vocab) for words in corpus]

In [54]:
sent_length=20

padded_rep = pad_sequences(one_hot_rep,padding='pre',maxlen=sent_length)

In [55]:
padded_rep

array([[   0,    0,    0, ..., 1821,  907,  550],
       [   0,    0,    0, ..., 3165, 3415,  959],
       [   0,    0,    0, ..., 2061, 4122, 3201],
       ...,
       [   0,    0,    0, ..., 3880, 3420, 2920],
       [   0,    0,    0, ..., 2077, 4811, 2198],
       [   0,    0,    0, ..., 3198, 4301, 1761]])

In [56]:
len(padded_rep)

18285

## Model

In [57]:
embedding_vector_features=40

model = Sequential()
model.add(Embedding(vocab,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())


Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
dropout_1 (Dropout)          (None, 20, 40)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dropout_2 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [58]:
len(padded_rep),y.shape

(18285, (18285,))

In [59]:
import numpy as np
X_final=np.array(padded_rep)
y_final=np.array(y)

In [60]:
X_final.shape,y_final.shape

((18285, 20), (18285,))

In [61]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(X_final,y_final,test_size=0.33,random_state=42)

In [62]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=10,batch_size=64)

C:\Users\yyash\anaconda3\envs\tensorgpu\lib\site-packages\tensorflow_core\python\framework\indexed_slices.py:433: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  "Converting sparse IndexedSlices to a dense Tensor of unknown shape. "


Train on 12250 samples, validate on 6035 samples
Epoch 1/10
12250/12250 [==============================] - 4s 319us/step - loss: 0.3430 - accuracy: 0.8345 - val_loss: 0.1934 - val_accuracy: 0.9167
Epoch 2/10
12250/12250 [==============================] - 3s 279us/step - loss: 0.1498 - accuracy: 0.9398 - val_loss: 0.1916 - val_accuracy: 0.9220
Epoch 3/10
12250/12250 [==============================] - 3s 280us/step - loss: 0.1085 - accuracy: 0.9604 - val_loss: 0.2024 - val_accuracy: 0.9205
Epoch 4/10
12250/12250 [==============================] - 3s 278us/step - loss: 0.0767 - accuracy: 0.9737 - val_loss: 0.2649 - val_accuracy: 0.9223
Epoch 5/10
12250/12250 [==============================] - 3s 278us/step - loss: 0.0552 - accuracy: 0.9819 - val_loss: 0.2611 - val_accuracy: 0.9221
Epoch 6/10
12250/12250 [==============================] - 3s 277us/step - loss: 0.0364 - accuracy: 0.9872 - val_loss: 0.3125 - val_accuracy: 0.9241
Epoch 7/10
12250/12250 [==============================] - 3s 27

### Performance Metrics And Accuracy

In [63]:

y_pred=model.predict_classes(X_test)

In [64]:
y_pred

array([[1],
       [0],
       [0],
       ...,
       [0],
       [1],
       [1]])

In [65]:
from sklearn.metrics import confusion_matrix, accuracy_score

In [66]:
confusion_matrix(y_test,y_pred)

array([[3084,  335],
       [ 174, 2442]], dtype=int64)

In [67]:
accuracy_score(y_test,y_pred)

0.9156586578293289

In [68]:
model.save("LSTM_Fake_new_classifier_model")